In [2]:
#Upload your kaggle API json file:
from google.colab import files
files.upload()

{}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d ashirwadsangwan/imdb-dataset

 99% 1.43G/1.44G [00:15<00:00, 109MB/s]
100% 1.44G/1.44G [00:15<00:00, 99.3MB/s]


In [5]:
#Define the path and name of the directory to be created:
#This is the path where we will extract the dataset:
import os
path = "/content/imdb-dataset"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
   print ("Successfully created the directory %s " % path)

Successfully created the directory /content/imdb-dataset 


In [6]:
#Extracts the Images from the Zip file downloaded to the path we created above:
from zipfile import ZipFile
file_name = "imdb-dataset.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall(path = "/content/imdb-dataset/")
  print('Done')

Done


In [7]:
!sudo apt-get update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]

In [8]:
!sudo apt-get install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 84 not upgraded.


In [9]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


In [10]:

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init("spark-3.1.2-bin-hadoop3.2")# SPARK_HOME
from pyspark.sql import SparkSession
import pyspark.sql.functions  as F
from pyspark.sql.functions import explode,split
from pyspark import SparkContext
 





In [11]:
sc=SparkContext()
session = SparkSession.builder.master("local[*]").getOrCreate()

In [12]:
df = session.read.csv("/content/imdb-dataset/title.akas.tsv", sep=r'\t', header=True).select("titleId","title")

In [13]:
df.show()

+---------+--------------------+
|  titleId|               title|
+---------+--------------------+
|tt0000001|Carmencita - span...|
|tt0000001|          Καρμενσίτα|
|tt0000001|          Карменсита|
|tt0000001|          Carmencita|
|tt0000001|          Carmencita|
|tt0000002|Le clown et ses c...|
|tt0000002|   A bohóc és kutyái|
|tt0000002|Le clown et ses c...|
|tt0000002|Clovnul si cainii...|
|tt0000002|  Клоун и его собаки|
|tt0000002|The Clown and His...|
|tt0000003|    Sarmanul Pierrot|
|tt0000003|     Szegény Pierrot|
|tt0000003|        Бедный Пьеро|
|tt0000003|      Pauvre Pierrot|
|tt0000003|        Poor Pierrot|
|tt0000003|      Pauvre Pierrot|
|tt0000004|         Un bon bock|
|tt0000004|      Un ţap de bere|
|tt0000004|         Un bon bock|
+---------+--------------------+
only showing top 20 rows



In [14]:
df.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- title: string (nullable = true)



In [15]:
df2 = session.read.csv("/content/imdb-dataset/name.basics.tsv", sep=r'\t', header=True)

In [16]:
df2.show()

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00531...|
|nm0000002|      Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0071877,tt01170...|
|nm0000003|    Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0054452,tt00491...|
|nm0000004|       John Belushi|     1949|     1982|actor,writer,soun...|tt0077975,tt00725...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0069467,tt00509...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,soundtrac...|tt0038109,tt00368...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,soundtrack,...|tt0043265,tt00338...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,sou

In [17]:
names= session.read.csv("/content/imdb-dataset/title.principals.tsv", sep=r'\t', header=True).sample(0.1)

In [18]:
names = names.where((names.category == "actor") | (names.category == "actress")).select("tconst","nconst")

In [19]:
names.show(truncate=False)

+---------+---------+
|tconst   |nconst   |
+---------+---------+
|tt0000007|nm0179163|
|tt0000008|nm0653028|
|tt0000014|nm0244989|
|tt0000017|nm3691272|
|tt0000026|nm1151424|
|tt0000032|nm3692479|
|tt0000064|nm0617588|
|tt0000076|nm0420198|
|tt0000109|nm0420198|
|tt0000111|nm0420198|
|tt0000132|nm0617588|
|tt0000174|nm1024447|
|tt0000230|nm0076933|
|tt0000246|nm0617588|
|tt0000310|nm0151209|
|tt0000335|nm1012612|
|tt0000351|nm0462306|
|tt0000376|nm0086082|
|tt0000376|nm0632577|
|tt0000381|nm0168621|
+---------+---------+
only showing top 20 rows



In [20]:
movie_names =df2.join(names,"nconst",'inner')

In [21]:
movie_names.show(truncate=False)

+---------+--------------+---------+---------+-------------------------+---------------------------------------+----------+
|nconst   |primaryName   |birthYear|deathYear|primaryProfession        |knownForTitles                         |tconst    |
+---------+--------------+---------+---------+-------------------------+---------------------------------------+----------+
|nm0000086|Louis de Funès|1914     |1983     |actor,writer,soundtrack  |tt0074103,tt0058135,tt0083109,tt0069747|tt0046243 |
|nm0000086|Louis de Funès|1914     |1983     |actor,writer,soundtrack  |tt0074103,tt0058135,tt0083109,tt0069747|tt0048994 |
|nm0000086|Louis de Funès|1914     |1983     |actor,writer,soundtrack  |tt0074103,tt0058135,tt0083109,tt0069747|tt0054394 |
|nm0000086|Louis de Funès|1914     |1983     |actor,writer,soundtrack  |tt0074103,tt0058135,tt0083109,tt0069747|tt0057414 |
|nm0000086|Louis de Funès|1914     |1983     |actor,writer,soundtrack  |tt0074103,tt0058135,tt0083109,tt0069747|tt0058135 |
|nm00000

In [22]:
splited = movie_names.select("primaryName","knownForTitles",split("knownForTitles",",").alias("split"))
exploded = splited.select("primaryName",explode("split").alias("titleId"))
exploded.show()

+--------------+---------+
|   primaryName|  titleId|
+--------------+---------+
|Louis de Funès|tt0074103|
|Louis de Funès|tt0058135|
|Louis de Funès|tt0083109|
|Louis de Funès|tt0069747|
|Louis de Funès|tt0074103|
|Louis de Funès|tt0058135|
|Louis de Funès|tt0083109|
|Louis de Funès|tt0069747|
|Louis de Funès|tt0074103|
|Louis de Funès|tt0058135|
|Louis de Funès|tt0083109|
|Louis de Funès|tt0069747|
|Louis de Funès|tt0074103|
|Louis de Funès|tt0058135|
|Louis de Funès|tt0083109|
|Louis de Funès|tt0069747|
|Louis de Funès|tt0074103|
|Louis de Funès|tt0058135|
|Louis de Funès|tt0083109|
|Louis de Funès|tt0069747|
+--------------+---------+
only showing top 20 rows



In [23]:
exploded.where("titleId == 'tt0117057'").show()

+----------------+---------+
|     primaryName|  titleId|
+----------------+---------+
|Barbra Streisand|tt0117057|
|Barbra Streisand|tt0117057|
|  Lesley Garrett|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
+----------------+---------+



In [24]:
prepared_df = exploded.join(df,"titleId","inner")

In [25]:
 prepared_df.where("title == 'Lucille Love: The Girl of Mystery'").show(truncate=False)

+---------+-------------+---------------------------------+
|titleId  |primaryName  |title                            |
+---------+-------------+---------------------------------+
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Burton Law   |Lucille Love: The Girl of Mystery|
|tt0004272|Burton Law   |Lucille Love: The Girl of Mystery|
|tt0004272|Burton Law   |Lucille Love: The Girl of Mystery|
|tt0004272|Burton Law   |Lucille Love: The Girl of Mystery|
|tt0004272|Burton Law   |Lucille Love: The Girl of Mystery|
|tt0004272|Burton Law   |Lucille Love: The Girl of Mystery|
|tt0004272|Jean Hathaway|Lucille Love: The Girl of Mystery|
|tt0004272|Jean Hathaway|Lucille Love: T

In [26]:
prepared_df.groupBy("title","primaryName").count().show()

+--------------------+-----------------+-----+
|               title|      primaryName|count|
+--------------------+-----------------+-----+
|Den gaadefulde Villa|        Hal Forde|    1|
|Dürfen wir schwei...|     Bella Polini|    2|
|Torchy Blane in P...|        Lola Lane|    8|
|      Sarlo Diktatör|       Eddie Dunn|    6|
|          A diktátor| Meredith Willson|    1|
|           O Ditador| Paulette Goddard|    3|
|  O Grande Escândalo|       Frank Orth|   10|
| Его девушка Пятница|       Helen Mack|    3|
|Dziewczyna Pietaszek|     Ernest Truex|   12|
|     Siste sensasjon|     Regis Toomey|   25|
|I doukissa kai o ...|    Árpád Lehotay|    1|
|      Saiturin joulu|  Ernest Thesiger|    1|
|Zumbis da Estrato...|      Wilson Wood|    1|
|Instinto Sanguinário|  Maureen Hingert|    1|
|O Nosso Agente em...|Yvonne Buckingham|    1|
|     Sankarin kunnia|      Bryan Hulme|    1|
|      En gång hjälte|      Bryan Hulme|    1|
|Mission spéciale ...|   Dominique Page|    1|
|Äventyraren 

In [27]:
basket  =prepared_df.groupBy("title").agg(F.collect_set("primaryName")).cache()

In [28]:
from pyspark.ml.fpm import FPGrowth
## modeling FP Growth alforithm to get Frequent Itemsets

fpGrowth = FPGrowth(itemsCol="collect_set(primaryName)", minSupport=0.0001, minConfidence=0.001)
model = fpGrowth.fit(basket)

In [29]:
model.freqItemsets.show(truncate= False)

+-------------------+----+
|items              |freq|
+-------------------+----+
|[Kevin Smith]      |144 |
|[Robert J. Wilke]  |132 |
|[Jennifer Lawrence]|126 |
|[Chiemi Ishimatsu] |109 |
|[Scott Spiegel]    |108 |
|[Åke Fridell]      |107 |
|[Jared Harris]     |122 |
|[Malcolm Dixon]    |119 |
|[Diane Kruger]     |116 |
|[Jay Baruchel]     |114 |
|[Conrad Goode]     |112 |
|[Simon Pegg]       |111 |
|[John Williams]    |332 |
|[Christian Simpson]|122 |
|[Miles Marsico]    |107 |
|[Amy Stiller]      |119 |
|[Brian Tochi]      |116 |
|[Pat McCormick]    |114 |
|[Andre Ware]       |112 |
|[David Bradley]    |282 |
+-------------------+----+
only showing top 20 rows



In [30]:
model.associationRules.show()

+--------------------+--------------------+----------+------------------+--------------------+
|          antecedent|          consequent|confidence|              lift|             support|
+--------------------+--------------------+----------+------------------+--------------------+
|[Eric Stuart, Iku...|    [Rodger Parsons]|       1.0| 7430.270833333333|1.317807504773547...|
|[Eric Stuart, Iku...|   [Michael Haigney]|       1.0|7482.2307692307695|1.317807504773547...|
|[Eric Stuart, Iku...|    [Rachael Lillis]|       1.0| 6645.708074534161|1.317807504773547...|
|[Eric Stuart, Iku...| [Chinami Nishimura]|       1.0| 6257.070175438596|1.317807504773547...|
|[Eric Stuart, Iku...|      [John Siegler]|       1.0| 5752.467741935484|1.317807504773547...|
|[Eric Stuart, Iku...|      [Mayumi Izuka]|       1.0| 5752.467741935484|1.317807504773547...|
|[Eric Stuart, Iku...|    [Rica Matsumoto]|       1.0| 5814.994565217392|1.317807504773547...|
|[Eric Stuart, Iku...|     [Inuko Inuyama]|       

In [31]:
lists = basket.select("collect_set(primaryName)").withColumnRenamed("collect_set(primaryName)","primary_names").sample(0.00001).collect()
sno_id_array = [ row.primary_names for row in lists]
sno_id_array

[['Mimis Stavrolemis'],
 ["Gloria D'Osvaldo"],
 ['Don Jaime de Mora y Aragón', 'Charles Baxter'],
 ['Jimmy Sireno'],
 ['Jorge Baião', 'Constantino Guimarães'],
 ['Richard Lornac', 'Myriam Nedellec'],
 ['Howard Hickman'],
 ['Wolfgang Müller', 'Werner Jacobs', 'Vico Torriani', 'Harry Tagore'],
 ['André Heinrich',
  'Fritz Hakl',
  'Joachim Hackethal',
  'Lech Grzmocinski',
  'Volker Schlöndorff',
  'Katharina Thalbach',
  'Wigand Witting',
  'Dietrich Frauboes'],
 ['Ben Bar', 'Mato Grbavac', 'Jerry Ryan'],
 ['Ashlee Spencer',
  'Eila Adams',
  'Charlie Atwell',
  'Amyka Lee',
  'Barbara Bella',
  'Chloe Toy',
  'Honour May',
  'Faye Taylor',
  'Brooke Little',
  'Kayla Louise',
  'Sammi Tye'],
 ['Nick Buck'],
 ['Kali Dreams', 'Punkin', 'Nadia Nasty']]

In [44]:
lblitems =sc.parallelize(sno_id_array)
wlitems=lblitems.flatMap(lambda line:line)

In [45]:
uniqueItems = wlitems.distinct()
supportRdd = wlitems.map(lambda item: (item , 1))

def sumOparator(x,y):
    return x+y

supportRdd = supportRdd.reduceByKey(sumOparator) 
supports = supportRdd.map(lambda item: item[1]) # Return only support values

In [46]:

minSupport = supports.min()

minSupport = 2 if minSupport == 1 else minSupport

supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport )

baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1])) 
print('1 . Table has crated...') 

supportRdd = supportRdd.map(lambda item: item[0])
supportRddCart = supportRdd

1 . Table has crated...


In [47]:

def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [48]:
c = 2 

while(supportRdd.isEmpty() == False):

    combined = supportRdd.cartesian(uniqueItems)
    combined = combined.map(lambda item: removeReplica(item))
  
    combined = combined.filter(lambda item: len(item) == c)
    combined = combined.distinct()

    
    combined_2 = combined.cartesian(lblitems)
    combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0]))
    
    combined_2 = combined_2.map(lambda item: item[0])
    combined_2 = combined_2.map(lambda item: (item , 1))
    combined_2 = combined_2.reduceByKey(sumOparator)
    combined_2 = combined_2.filter(lambda item: item[1] >= minSupport)

    baseRdd = baseRdd.union(combined_2)
    
    combined_2 = combined_2.map(lambda item: item[0])
    supportRdd = combined_2
    print(c ,'. Table has crated... ')
    c = c+1 

In [37]:
class Filter():

    def __init__(self):
        
        self.stages = 1


    def filterForConf(self, item , total):
        
        if(len(item[0][0]) > len(item[1][0])  ):
            if(self.checkItemSets(item[0][0] , item[1][0]) == False):
                pass
            else:
                return (item)       
        else:
            pass  
        self.stages = self.stages + 1

    def checkItemSets(self, item_1 , item_2):

        if(len(item_1) > len(item_2)):
            return all(any(k == l for k in item_1 ) for l in item_2)
        else:
            return all(any(k == l for k in item_2 ) for l in item_1)


    def calculateConfidence(self, item):

        parent = set(item[0][0])
        
        if(isinstance(item[1][0] , str)):
            child  = set([item[1][0]])
        else:
            child  = set(item[1][0])
        
        parentSupport = item[0][1]
        childSupport = item[1][1]
       

        support = (parentSupport / childSupport)*100

        return list([ list(child) ,  list(parent.difference(child)) , support ])

In [51]:
calcuItems = baseRdd.cartesian(baseRdd)
ff = Filter()
total = calcuItems.count()

In [52]:
print('# : Aggregated support values preparing for the confidence calculatations')
baseRddConfidence = calcuItems.filter(lambda item: ff.filterForConf(item , total))

# : Aggregated support values preparing for the confidence calculatations


In [53]:
print('# : Aggregated support values are ready !')
baseRddConfidence = baseRddConfidence.map(lambda item: ff.calculateConfidence(item))

# : Aggregated support values are ready !


In [54]:
print(baseRddConfidence.collect())
## No confidence could be made in current use case 

[]
